In [1]:
import os
import string
import unicodedata
from datetime import datetime
from pprint import pprint

import torch
import torchaudio
from tqdm import tqdm
from underthesea import sent_tokenize
from unidecode import unidecode

from vinorm import TTSnorm
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts

device = "cuda:0"

xtts_checkpoint = "checkpoints/GPT_XTTS_FT-August-30-2024_08+19AM-6a6b942/best_model_99875.pth"
xtts_config = "checkpoints/GPT_XTTS_FT-August-30-2024_08+19AM-6a6b942/config.json"
# xtts_checkpoint = "checkpoints/GPT_XTTS_FT-August-29-2024_07+13AM-5e9900c/best_model.pth"
# xtts_config = "checkpoints/GPT_XTTS_FT-August-29-2024_07+13AM-5e9900c/config.json"
xtts_vocab = "checkpoints/XTTS_v2.0_original_model_files/vocab.json"

config = XttsConfig()
config.load_json(xtts_config)
XTTS_MODEL = Xtts.init_from_config(config)
print("Loading XTTS model! ")
XTTS_MODEL.load_checkpoint(config,
                            checkpoint_path=xtts_checkpoint,
                            vocab_path=xtts_vocab,
                            use_deepspeed=False)

st3 = torch.load("checkpoints/XTTS_v2.0_original_model_files/model.pth", map_location="cpu")
hifigan_checkpoint = st3["model"]
keys = list(hifigan_checkpoint.keys())
for key in keys:
    if "hifigan_decoder." in key:
        new_key = key.replace("hifigan_decoder.", "")
        hifigan_checkpoint[new_key] = hifigan_checkpoint[key]
        del hifigan_checkpoint[key]
    else:
        del hifigan_checkpoint[key]
XTTS_MODEL.hifigan_decoder.load_state_dict(hifigan_checkpoint, strict=True)
del hifigan_checkpoint


if torch.cuda.is_available():
    XTTS_MODEL.to(device)
print("Load successfully!")

Loading XTTS model! 


/mnt/anhnh/TTS/XTTS-3-training-stages/TTS/utils/io.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_location, **kwargs)
/tmp/ipyker

Load successfully!


In [2]:
# from vinorm import TTSnorm

def calculate_keep_len(text):
    word_count = len(text.split())
    num_punct = (
        text.count(".")
        + text.count("!")
        + text.count("?")
        + text.count(",")
    )

    if word_count < 5:
        return 15000 * word_count + 2000 * num_punct
    elif word_count < 10:
        return 13000 * word_count + 2000 * num_punct
    return -1


def normalize_vietnamese_text(text):
    # last = text[-1]
    # text = text[:-1]
    # text = text.replace(".", ",").replace("?", ",").replace("!", ",")
    # if text[-1] == ",":
    #     text = text[:-1] + "."
    # text = text + last
    text = (
        TTSnorm(text, unknown=False, lower=False, rule=True)
        .replace("..", ".")
        .replace("!.", "!")
        .replace("?.", "?")
        .replace(" .", ".")
        .replace(" ,", ",")
        .replace('"', "")
        .replace("'", "")
        .replace("AI", "Ây Ai")
        .replace("A.I", "Ây Ai")
        .replace("Hmm", "")
    )
    # text = text.replace("?", ".").replace("!", ".")
    return text

In [3]:
import numpy as np

def adjust_audio(data, sample_rate, pitch=0, speed=1):
    # Adjust pitch
    if pitch != 0:
        # Calculate the new sample rate for pitch change
        new_sample_rate = int(sample_rate * (2.0 ** (pitch / 12.0)))
        # Resample the data to new sample rate for pitch adjustment
        duration = len(data) / sample_rate
        time_old = np.linspace(0, duration, len(data))
        time_new = np.linspace(0, duration, int(len(data) * (new_sample_rate / sample_rate)))
        data = np.interp(time_new, time_old, data)
        # sample_rate = new_sample_rate
    
    # Adjust speed
    if speed != 1:
        # Resample data to the new speed
        duration = len(data) / sample_rate
        time_old = np.linspace(0, duration, len(data))
        time_new = np.linspace(0, duration * speed, len(data))
        data = np.interp(time_new, time_old, data)
    
    return data

In [4]:
from IPython.display import Audio

In [5]:
def get_file_name(text, max_char=50):
    filename = text[:max_char]
    filename = filename.lower()
    filename = filename.replace(" ", "_")
    filename = filename.translate(str.maketrans("", "", string.punctuation.replace("_", "")))
    filename = unidecode(filename)
    current_datetime = datetime.now().strftime("%m%d%H%M%S")
    filename = f"{current_datetime}_{filename}"
    return filename

def run_tts(XTTS_MODEL, lang, tts_text, speaker_audio_file,
            normalize_text= True,
            verbose=False,
            output_chunks=False,
            stream=False):
    """
    Run text-to-speech (TTS) synthesis using the provided XTTS_MODEL.

    Args:
        XTTS_MODEL: A pre-trained TTS model.
        lang (str): The language of the input text.
        tts_text (str): The text to be synthesized into speech.
        speaker_audio_file (str): Path to the audio file of the speaker to condition the synthesis on.
        normalize_text (bool, optional): Whether to normalize the input text. Defaults to True.
        verbose (bool, optional): Whether to print verbose information. Defaults to False.
        output_chunks (bool, optional): Whether to save synthesized speech chunks separately. Defaults to False.

    Returns:
        str: Path to the synthesized audio file.
    """

    if XTTS_MODEL is None or not speaker_audio_file:
        return "You need to run the previous step to load the model !!", None, None

    output_dir = "./output"
    os.makedirs(output_dir, exist_ok=True)

    gpt_cond_latent, speaker_embedding = XTTS_MODEL.get_conditioning_latents(
        audio_path=speaker_audio_file,
        gpt_cond_len=XTTS_MODEL.config.gpt_cond_len,
        max_ref_length=XTTS_MODEL.config.max_ref_len,
        sound_norm_refs=XTTS_MODEL.config.sound_norm_refs,
    )

    if normalize_text and lang == "vi":
        # Bug on google colab
        tts_text = normalize_vietnamese_text(tts_text)


    if lang in ["ja", "zh-cn"]:
        tts_texts = tts_text.split("。")
    else:
        tts_texts = sent_tokenize(tts_text)
        # tts_texts = [tts_text]

    input_text = []
    chunk_i = ""
    len_chunk_i = 0
    for sentence in tts_texts:
        chunk_i += " " + sentence
        len_chunk_i += len(sentence.split())
        if len_chunk_i > 25:
            input_text.append(chunk_i.strip())
            chunk_i = ""
            len_chunk_i = 0

    if (len(input_text) > 0) and (len_chunk_i < 15):
        input_text[-1] += chunk_i
    else:
        input_text.append(chunk_i)

    if verbose:
        print("Text for TTS:")
        pprint(tts_texts)

    wav_chunks = []
    for text in tqdm(input_text):
        if text.strip() == "":
            continue
        if stream:
            chunks = [torch.tensor(adjust_audio(x.cpu().numpy(), 24000, -5)) for x in XTTS_MODEL.inference_stream(
                text=text,
                language=lang,
                gpt_cond_latent=gpt_cond_latent,
                speaker_embedding=speaker_embedding,
                temperature=0.5,
                length_penalty=1.0,
                repetition_penalty=10.0,
                top_k=10,
                top_p=0.3,
            )]

            # chunks = [x for x in XTTS_MODEL.inference_stream(
            #     text=text,
            #     language=lang,
            #     gpt_cond_latent=gpt_cond_latent,
            #     speaker_embedding=speaker_embedding,
            #     temperature=0.5,
            #     length_penalty=1.0,
            #     repetition_penalty=10.0,
            #     top_k=10,
            #     top_p=0.3,
            # )]

            print(f"Num chunks: {len(chunks)}")

            wav_chunk = torch.cat(chunks, dim=0)
            # wav_chunk = torch.tensor(process_and_concatenate_chunks(chunks, 24000, 10, 50, 50))

            wav_chunks.append(wav_chunk)
        else:
            wav_chunk = XTTS_MODEL.inference(
                text=text,
                language=lang,
                gpt_cond_latent=gpt_cond_latent,
                speaker_embedding=speaker_embedding,
                temperature=0.1,
                length_penalty=1.0,
                repetition_penalty=10.0,
                top_k=10,
                top_p=0.3,
            )

            wav_chunk["wav"] = torch.tensor(wav_chunk["wav"])

            # wav_chunks.append(torch.tensor(chipmunk_voice(wav_chunk["wav"].cpu().numpy(), 24000, 10)))
            wav_chunks.append(wav_chunk["wav"])

    out_wav = torch.cat(wav_chunks, dim=0).unsqueeze(0).cpu()
    # out_path = os.path.join(output_dir, f"{get_file_name(tts_text)}.wav")
    # tempfile = tempfile.NamedTemporaryFile(delete=False, suffix='.mp3').name
    # torchaudio.save(out_path, out_wav, 22000)

    # if verbose:
    #     print(f"Saved final file to {out_path}")

    return out_wav


# @markdown Chọn ngôn ngữ:
language = "Tiếng Việt" # @param ["Tiếng Việt", "Tiếng Anh","Tiếng Tây Ban Nha", "Tiếng Pháp","Tiếng Đức","Tiếng Ý", "Tiếng Bồ Đào Nha", "Tiếng Ba Lan", "Tiếng Thổ Nhĩ Kỳ", "Tiếng Nga", "Tiếng Hà Lan", "Tiếng Séc", "Tiếng Ả Rập", "Tiếng Trung (giản thể)", "Tiếng Nhật", "Tiếng Hungary", "Tiếng Hàn", "Tiếng Hindi"]
# @markdown Văn bản để đọc. Độ dài tối thiểu mỗi câu nên từ 10 từ để đặt kết quả tốt nhất.
input_text ="To call this endpoint using Python requests and display the response with IPython, we'll assume the FastAPI server is running on localhost and the endpoint is active. Here's how you can do it." # @param {type:"string"}
input_text = """Đang ngủ say thì bất ngờ có tiếng đập cửa, Hùng vội mặc tạm một chiếc áo vào rồi chạy ra mở cửa. Mọi khi vào tầm này chả thấy ai gọi cửa cả nên hùng nghĩ chắc là có chuyện gấp nên hắn ta chạy ra thật nhanh để mở.

Lúc mở cửa ra thì Hùng ngỡ ngàng khi thấy có một mảnh giấy để ở cửa ghi dòng chữ “xin lỗi vì đã làm phiền”.

Bực Mình Hùng chả nói chả rằng mà chỉ biết đi vào nhà và đóng cửa lại rồi tiếp tục ngủ nhưng đến gần sáng thì lại có một tiếng đập cửa rất mạnh kèm theo tiếng nói khá thảm thiết:” ối giời ơi mở cửa nhanh lên không mở là không kịp đâu……..!”

Lúc này Hùng bực mình lao ra một tay cầm gậy tay kia mở cửa. Khi mở Cửa ra thì Hùng lại chả nhìn thấy gì…mà chỉ thấy 1 dòng chữ ghi trên giấy:” Các hạ đã ra muộn hẹn tái ngộ lần sau…” Địt con chị nó Hùng bực mình lắm mà không có cách nào khắc phục cả."""
# input_text = "I'm doing great! How about you? My girl friend cheated on me."
# input_text = "Okie!  Bạn ở khu vực nào vậy?  Chi sẽ tìm quán ngon gần bạn nhất."

# input_text = "Tớ không thể tưởng tượng được là tại sao trên đời này lại có người tuyệt vời như cậu vậy nhỉ? Ôi không! Có chuyện gì vậy? Cậu có sao không?"
# input_text = "Đã có ai đái vào bát cơm của bạn vào sáng nay à?... Tớ không thể tưởng tượng được là tại sao trên đời này lại có người tuyệt vời như cậu vậy nhỉ?"
# @markdown Chọn giọng mẫu:
# input_text = "Mạnh hiểu bạn đang muốn nói về một điều gì đó mang tính gợi cảm.  Tuy nhiên, Mạnh là một mô hình ngôn ngữ được thiết kế để cung cấp thông tin và hỗ trợ một cách lịch sự và phù hợp.  \n\nMạnh không thể cung cấp nội dung khiêu dâm.  Bạn có muốn Mạnh giúp bạn tìm kiếm thông tin về một chủ đề khác không?  Mạnh có thể chia sẻ những câu chuyện thú vị, giải đáp những thắc mắc của bạn về khoa học, lịch sử hoặc bất cứ điều gì bạn quan tâm."
# input_text = "Ừ thì... hồi nào đó, tớ đi siêu thị mua mì gói, mà quên mang ví.    Bỗng nhiên, một ông già đi ngang qua, nhìn tớ vẻ mặt tội nghiệp, hỏi: \"Con ơi, thiếu gì?\".  Tớ ngại ngùng nói: \"Thưa ông, cháu quên mang ví\".  Ông già cười hiền và nói: \"Thôi, ông mua cho con một gói mì gói ngon nhất nhé!\".    Tớ mừng hú vía, cảm ơn ông già rối rít.  Ông ấy còn bảo: \"Mì gói ngon nhất là mì gói được chia sẻ\".  Tớ cười không ngậm được mồm, cảm động lắm!    Bạn có chuyện vui nào muốn chia sẻ không?"
# input_text = "Chào bạn! Mạnh đây, vui lắm được gặp bạn.    Bạn có chuyện gì muốn tâm sự không?"
# input_text = "Ôi, đi làm áp lực thì ai mà chẳng chán!   Mèo Mèo hiểu mà!  Cứ nghĩ đến việc phải thức dậy sớm, tắc đường, rồi deadline cứ đuổi theo, thôi rồi!    Bạn muốn tâm sự về công việc gì?  Mèo Mèo có thể giúp bạn phân tích vấn đề, hoặc đơn giản là cùng bạn than thở cho đỡ bức bối!"
reference_audio = "datasets/wavs/audio_1377.08_1383.66.wav" #@VuTruNguyenThuy
# reference_audio = "datasets/wavs/audio_8.75_17.03.wav" #@ThahPahm
reference_audio = "datasets/wavs/audio_166.18_172.4.wav" #@ThePresentWriter
# reference_audio = "datasets/wavs/audio_446.56_452.42.wav" #@chauanhchao
# reference_audio = "datasets/wavs/audio_703.28_710.74.wav" #@ducisreal

# @markdown Tự động chuẩn hóa chữ (VD: 20/11 -> hai mươi tháng mười một)
normalize_text = True # @param {type:"boolean"}
# @markdown In chi tiết xử lý
verbose = True # @param {type:"boolean"}
# @markdown Lưu từng câu thành file riêng lẻ.
output_chunks = True # @param {type:"boolean"}

In [12]:
input_text = "Em không hiểu sao anh lại nói vậy. Có phải là em làm gì sai không? Em chỉ muốn được ở bên anh, được chia sẻ những khoảng khắc đẹp với anh mà thôi. Anh có thể nói cho em biết anh đang nghĩ gì không? Em rất muốn hiểu anh hơn."
# input_text = "Xin chào"

audio_file = run_tts(XTTS_MODEL,
        lang="vi",
        tts_text=input_text,
        speaker_audio_file=reference_audio,
        normalize_text=normalize_text,
        verbose=verbose,
        output_chunks=output_chunks,
        stream=False)

# Audio(chipmunk_voice(audio_file.numpy(), 24000, 10), rate=24000)
Audio(audio_file, rate=24000)

Text for TTS:
['Em không hiểu sao anh lại nói vậy.',
 'Có phải là em làm gì sai không?',
 'Em chỉ muốn được ở bên anh, được chia sẻ những khoảng khắc đẹp với anh mà '
 'thôi.',
 'Anh có thể nói cho em biết anh đang nghĩ gì không?',
 'Em rất muốn hiểu anh hơn.']


  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:02<00:02,  2.01s/it]

tensor([[1006, 1011,  952,  879,  333,  325,  577,   28,   99,  627,  880,   23,
          280,  635,  256,  290,  299,   74,  703,  140,  468,  481,  131,  422,
          405,  487,  908,  229,  836,  153,  424,  841,  848,  876,   64,  158,
          266,  758,  421,  767,  581,  685,  866,  730,  137,  395,  384,  231,
          625,  655,  573,  504,  430,   40,  327,  432,  614,  923,  837,  649,
          645,  615,   82,  296,  678,   71,  482,  145,  183,  794,  638,  128,
          139,  157,  566,  275,  872,  277,  407,  630,   25,   59,  160,  527,
          702,  592,  792,  613,  524,  554,  508,  650,  510,  479,  628,  163,
           11,  522,  538,  534,  180,   45,  316,  727,  632,  681,  584,  189,
          873,  594,  461,   66,  477,   90,  356,  201,  776,  434,  391,  775,
          221,  415,  802,  869,  821, 1025]], device='cuda:0')


100%|██████████| 2/2 [00:03<00:00,  1.75s/it]

tensor([[1006, 1011,  952,  879,  201,  508,  685,  577,   23,  139,  662,  325,
          758,  542,  876,  313,  423,  266,  422,  384,   71,  438,  703,  767,
          573,  702,  468,   28,  128,  228,  549,  602,  871,  369,  421,  424,
          584,  655,  632,  144,  527,  137,  229,  269,   99,   81,   40,  477,
          730,  807,  635,  256,  487,  301,  391,  828,  614,  174,  680,  649,
          504,  645,  615,  628,  440,  605,  333,  762,  534,  794,  327,  131,
          522,   90,  681,   17,  627,  880,  663,  373,  706,  277,  407,  559,
          414,  630,  349,   74,   82,  296,  653,  835,  808,  802, 1025]],
       device='cuda:0')


In [6]:
input_text = "Em không hiểu sao anh lại nói vậy. Có phải là em làm gì sai không? Em chỉ muốn được ở bên anh, được chia sẻ những khoảng khắc đẹp với anh mà thôi. Anh có thể nói cho em biết anh đang nghĩ gì không? Em rất muốn hiểu anh hơn."
# input_text = "Bạn có chuyện gì muốn chia sẻ không?"

audio_file = run_tts(XTTS_MODEL,
        lang="vi",
        tts_text=input_text,
        speaker_audio_file=reference_audio,
        normalize_text=normalize_text,
        verbose=verbose,
        output_chunks=output_chunks,
        stream=False)

# Audio(chipmunk_voice(audio_file.numpy(), 24000, 10), rate=24000)
Audio(audio_file, rate=24000)

Text for TTS:
['Em không hiểu sao anh lại nói vậy.',
 'Có phải là em làm gì sai không?',
 'Em chỉ muốn được ở bên anh, được chia sẻ những khoảng khắc đẹp với anh mà '
 'thôi.',
 'Anh có thể nói cho em biết anh đang nghĩ gì không?',
 'Em rất muốn hiểu anh hơn.']


 50%|█████     | 1/2 [00:02<00:02,  2.80s/it]

tensor([[1006, 1011,  952,  879,  333,  325,  577,   28,   99,  627,  880,   23,
          280,  635,  256,  290,  299,   74,  703,  140,  468,  481,  131,  422,
          405,  487,  908,  229,  836,  153,  424,  841,  848,  876,   64,  158,
          266,  758,  421,  767,  581,  685,  866,  730,  137,  395,  384,  231,
          625,  655,  573,  504,  430,   40,  327,  432,  614,  923,  837,  649,
          645,  615,   82,  296,  678,   71,  482,  145,  183,  794,  638,  128,
          139,  157,  566,  275,  872,  277,  407,  630,   25,   59,  160,  527,
          702,  592,  792,  613,  524,  554,  508,  650,  510,  479,  628,  163,
           11,  522,  538,  534,  180,   45,  316,  727,  632,  681,  584,  189,
          873,  594,  461,   66,  477,   90,  356,  201,  776,  434,  391,  775,
          221,  415,  802,  869,  821, 1025]], device='cuda:0')


100%|██████████| 2/2 [00:04<00:00,  2.23s/it]

tensor([[1006, 1011,  952,  879,  201,  508,  685,  577,   23,  139,  662,  325,
          758,  542,  876,  313,  423,  266,  422,  384,   71,  438,  703,  767,
          573,  702,  468,   28,  128,  228,  549,  602,  871,  369,  421,  424,
          584,  655,  632,  144,  527,  137,  229,  269,   99,   81,   40,  477,
          730,  807,  635,  256,  487,  301,  391,  828,  614,  174,  680,  649,
          504,  645,  615,  628,  440,  605,  333,  762,  534,  794,  327,  131,
          522,   90,  681,   17,  627,  880,  663,  373,  706,  277,  407,  559,
          414,  630,  349,   74,   82,  296,  653,  835,  808,  802, 1025]],
       device='cuda:0')


In [1]:
import torch

# st1 = torch.load("checkpoints/GPT_XTTS_FT-August-29-2024_07+13AM-5e9900c/best_model.pth", map_location="cpu")
# st2 = torch.load("checkpoints/GPT_XTTS_FT-August-30-2024_08+19AM-6a6b942/best_model.pth", map_location="cpu")

In [2]:
st3 = torch.load("checkpoints/XTTS_v2.0_original_model_files/model.pth", map_location="cpu")

/tmp/ipykernel_161802/298480634.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  st3 = torch.load("checkpoints/XTTS_v2.0_original_model_files/model.pth", map_location="cp

In [3]:
st3["model"].keys()

odict_keys(['mel_stats', 'gpt.conditioning_encoder.init.weight', 'gpt.conditioning_encoder.init.bias', 'gpt.conditioning_encoder.attn.0.norm.weight', 'gpt.conditioning_encoder.attn.0.norm.bias', 'gpt.conditioning_encoder.attn.0.qkv.weight', 'gpt.conditioning_encoder.attn.0.qkv.bias', 'gpt.conditioning_encoder.attn.0.proj_out.weight', 'gpt.conditioning_encoder.attn.0.proj_out.bias', 'gpt.conditioning_encoder.attn.1.norm.weight', 'gpt.conditioning_encoder.attn.1.norm.bias', 'gpt.conditioning_encoder.attn.1.qkv.weight', 'gpt.conditioning_encoder.attn.1.qkv.bias', 'gpt.conditioning_encoder.attn.1.proj_out.weight', 'gpt.conditioning_encoder.attn.1.proj_out.bias', 'gpt.conditioning_encoder.attn.2.norm.weight', 'gpt.conditioning_encoder.attn.2.norm.bias', 'gpt.conditioning_encoder.attn.2.qkv.weight', 'gpt.conditioning_encoder.attn.2.qkv.bias', 'gpt.conditioning_encoder.attn.2.proj_out.weight', 'gpt.conditioning_encoder.attn.2.proj_out.bias', 'gpt.conditioning_encoder.attn.3.norm.weight', 'gpt

In [15]:
st2["model"].keys()

odict_keys(['xtts.mel_stats', 'xtts.hifigan_decoder.waveform_decoder.conv_pre.bias', 'xtts.hifigan_decoder.waveform_decoder.conv_pre.weight', 'xtts.hifigan_decoder.waveform_decoder.ups.0.bias', 'xtts.hifigan_decoder.waveform_decoder.ups.0.parametrizations.weight.original0', 'xtts.hifigan_decoder.waveform_decoder.ups.0.parametrizations.weight.original1', 'xtts.hifigan_decoder.waveform_decoder.ups.1.bias', 'xtts.hifigan_decoder.waveform_decoder.ups.1.parametrizations.weight.original0', 'xtts.hifigan_decoder.waveform_decoder.ups.1.parametrizations.weight.original1', 'xtts.hifigan_decoder.waveform_decoder.ups.2.bias', 'xtts.hifigan_decoder.waveform_decoder.ups.2.parametrizations.weight.original0', 'xtts.hifigan_decoder.waveform_decoder.ups.2.parametrizations.weight.original1', 'xtts.hifigan_decoder.waveform_decoder.ups.3.bias', 'xtts.hifigan_decoder.waveform_decoder.ups.3.parametrizations.weight.original0', 'xtts.hifigan_decoder.waveform_decoder.ups.3.parametrizations.weight.original1', 'x

In [19]:
st1["model"]["xtts.hifigan_decoder.waveform_decoder.conv_pre.weight"][0][0]

tensor([-0.0326, -0.0191, -0.0129, -0.0205, -0.0223, -0.0245, -0.0249])

In [20]:
st2["model"]["xtts.hifigan_decoder.waveform_decoder.conv_pre.weight"][0][0]

tensor([-0.0090, -0.0083, -0.0060,  0.0118, -0.0103,  0.0105,  0.0054])

In [21]:
st3["model"]["hifigan_decoder.waveform_decoder.conv_pre.weight"][0][0]

tensor([-0.0326, -0.0191, -0.0129, -0.0205, -0.0223, -0.0245, -0.0249])

In [1]:
from TTS.tts.layers.xtts.trainer.gpt_trainer import GPTArgs, GPTTrainer

model = GPTTrainer()

TypeError: GPTTrainer.__init__() missing 1 required positional argument: 'config'

In [1]:
import os
import gc

from trainer import Trainer, TrainerArgs

from TTS.config.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.layers.xtts.trainer.gpt_trainer import GPTArgs, GPTTrainer, GPTTrainerConfig, XttsAudioConfig
from TTS.utils.manage import ModelManager

from dataclasses import dataclass, field
from typing import Optional
from transformers import HfArgumentParser


output_path="checkpoints/"
train_csv="large-datasets/metadata_train.csv"
eval_csv="large-datasets/metadata_eval.csv"
language="vi"
num_epochs=5
batch_size=8
grad_acumm=2
max_text_length=250
max_audio_length=255995
weight_decay=1e-2
lr=5e-6

RUN_NAME = "GPT_XTTS_FT"
PROJECT_NAME = "XTTS_trainer"
DASHBOARD_LOGGER = "tensorboard"
LOGGER_URI = None

# Set here the path that the checkpoints will be saved. Default: ./run/training/
# OUT_PATH = os.path.join(output_path, "run", "training")
OUT_PATH = output_path

# Training Parameters
OPTIMIZER_WD_ONLY_ON_WEIGHTS = True  # for multi-gpu training please make it False
START_WITH_EVAL = False  # if True it will star with evaluation
BATCH_SIZE = batch_size  # set here the batch size
GRAD_ACUMM_STEPS = grad_acumm  # set here the grad accumulation steps


# Define here the dataset that you want to use for the fine-tuning on.
config_dataset = BaseDatasetConfig(
    formatter="coqui",
    dataset_name="ft_dataset",
    path=os.path.dirname(train_csv),
    meta_file_train=os.path.basename(train_csv),
    meta_file_val=os.path.basename(eval_csv),
    language=language,
)

# Add here the configs of the datasets
DATASETS_CONFIG_LIST = [config_dataset]

# Define the path where XTTS v2.0.1 files will be downloaded
CHECKPOINTS_OUT_PATH = os.path.join(OUT_PATH, "XTTS_v2.0_original_model_files/")
os.makedirs(CHECKPOINTS_OUT_PATH, exist_ok=True)


# DVAE files
DVAE_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/dvae.pth"
MEL_NORM_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/mel_stats.pth"

# Set the path to the downloaded files
DVAE_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(DVAE_CHECKPOINT_LINK))
MEL_NORM_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(MEL_NORM_LINK))

# download DVAE files if needed
if not os.path.isfile(DVAE_CHECKPOINT) or not os.path.isfile(MEL_NORM_FILE):
    print(" > Downloading DVAE files!")
    ModelManager._download_model_files([MEL_NORM_LINK, DVAE_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True)


# Download XTTS v2.0 checkpoint if needed
TOKENIZER_FILE_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/vocab.json"
XTTS_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/model.pth"
XTTS_CONFIG_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/config.json"

# XTTS transfer learning parameters: You we need to provide the paths of XTTS model checkpoint that you want to do the fine tuning.
TOKENIZER_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(TOKENIZER_FILE_LINK))  # vocab.json file
XTTS_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(XTTS_CHECKPOINT_LINK))  # model.pth file
XTTS_CONFIG_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(XTTS_CONFIG_LINK))  # config.json file

# download XTTS v2.0 files if needed
if not os.path.isfile(TOKENIZER_FILE) or not os.path.isfile(XTTS_CHECKPOINT):
    print(" > Downloading XTTS v2.0 files!")
    ModelManager._download_model_files(
        [TOKENIZER_FILE_LINK, XTTS_CHECKPOINT_LINK, XTTS_CONFIG_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True
    )

# init args and config
model_args = GPTArgs(
    max_conditioning_length=132300,  # 6 secs
    min_conditioning_length=11025,  # 0.5 secs
    debug_loading_failures=False,
    max_wav_length=max_audio_length,  # ~11.6 seconds
    max_text_length=max_text_length,
    mel_norm_file=MEL_NORM_FILE,
    dvae_checkpoint=DVAE_CHECKPOINT,
    xtts_checkpoint=XTTS_CHECKPOINT,  # checkpoint path of the model that you want to fine-tune
    tokenizer_file=TOKENIZER_FILE,
    gpt_num_audio_tokens=1026,
    gpt_start_audio_token=1024,
    gpt_stop_audio_token=1025,
    gpt_use_masking_gt_prompt_approach=True,
    gpt_use_perceiver_resampler=True,
)
# define audio config
audio_config = XttsAudioConfig(sample_rate=22050, dvae_sample_rate=22050, output_sample_rate=24000)
# training parameters config

config = GPTTrainerConfig()

config.load_json(XTTS_CONFIG_FILE)

config.epochs = num_epochs
config.output_path = OUT_PATH
config.model_args = model_args
config.run_name = RUN_NAME
config.project_name = PROJECT_NAME
config.run_description = """
    GPT XTTS training
    """,
config.dashboard_logger = DASHBOARD_LOGGER
config.logger_uri = LOGGER_URI
config.audio = audio_config
config.batch_size = BATCH_SIZE
config.num_loader_workers = 8
config.eval_split_max_size = 256
config.print_step = 50
config.plot_step = 100
config.log_model_step = 100
config.save_step = 50000
config.save_n_checkpoints = 1
config.save_checkpoints = True
config.print_eval = False
config.optimizer = "AdamW"
config.optimizer_wd_only_on_weights = OPTIMIZER_WD_ONLY_ON_WEIGHTS
config.optimizer_params = {"betas": [0.9, 0.96], "eps": 1e-8, "weight_decay": weight_decay}
config.lr = lr
config.lr_scheduler = "MultiStepLR"
config.lr_scheduler_params = {"milestones": [50000 * 18, 150000 * 18, 300000 * 18], "gamma": 0.5, "last_epoch": -1}
config.test_sentences = []

# init the model from config
model = GPTTrainer.init_from_config(config)

/mnt/anhnh/TTS/XTTS-3-training-stages/TTS/utils/io.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_location, **kwargs)


 > Loading checkpoint with 863 additional tokens.


/mnt/anhnh/TTS/XTTS-3-training-stages/TTS/tts/layers/tortoise/arch_utils.py:336: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.mel_norms = torch.load(f)


>> DVAE weights restored from: checkpoints/XTTS_v2.0_original_model_files/dvae.pth


/mnt/anhnh/TTS/XTTS-3-training-stages/TTS/tts/layers/xtts/trainer/gpt_trainer.py:192: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dvae_checkpoint = torch.load(self.args.dv

In [4]:
model.xtts.gpt.text_embedding.parameters()

<bound method Module.parameters of Embedding(7544, 1024)>